### --- Assume all company submitted; assume all blank columns and length of 1; assume "0.2x size" format wont change; assume that one tradetype has only one number for size calculating (such as "10", "5")

In [131]:
import pandas as pd
import numpy as np
import xlrd

In [132]:
sheets_dict = pd.read_excel("Input_File.xlsx", sheet_name=None)
workbook = xlrd.open_workbook("Input_File.xlsx")

In [133]:
sheet_names = workbook.sheet_names()
worksheets = {}
for i in range(len(sheet_names)-1):
    worksheets[sheet_names[i+1]] = workbook.sheet_by_name(sheet_names[i+1])

In [134]:
def check_not_empty(worksheet,row,col):
        return worksheet.cell(row,col).value != ''

def check_whole_col_empty(worksheet,tot_rows,col):
    result = True
    for i in range(tot_rows):
        if check_not_empty(worksheet,i,col):
            result = False
            break;
    return result;

def combine_consecutive_number(alist):
    seat_need_delete = []
    for i in range(len(alist)-1):
        if alist[i+1] == alist[i]+1:
            seat_need_delete.append(alist[i])
    result = list(set(alist)-set(seat_need_delete))
    result.sort()
    return result

def str_contains(str1,str2,str3='yuezhou',str4='yuezhou'):
    result = False
    if str2 in str1:
        result = True
    if str3 in str1:
        result = True
    if str4 in str1:
        result = True
    return result;

def get_size_from_string(astring):
    astring = astring.replace('x','')
    return float(astring.split()[0])

### Get all blank rows and cols

In [135]:
worksheet_now = worksheets[sheet_names[1]]
n_cols = worksheet_now.ncols
n_rows = worksheet_now.nrows

effective_cols =[]
effective_rows = []

for i in range(n_cols):
    if not check_whole_col_empty(worksheet_now,n_rows,i):
        effective_cols.append(i)
        
empty_cols = list(set(range(0,n_cols))-set(effective_cols))
empty_cols.sort()        

# attention: first 2 in the effective cols are "median" and "bis/ask spread for a trade"
for i in range(n_rows):
    if check_not_empty(worksheet_now,i,empty_cols[0]+1):
        effective_rows.append(i)
        
above_table_empty_rows = list(set(range(effective_rows[0],effective_rows[-1]))-set(effective_rows))
above_table_empty_rows.sort()
above_table_empty_rows = combine_consecutive_number(above_table_empty_rows)

### get CCY --list

In [136]:
currency = worksheet_now.cell(4,0).value
currency

'USD'

### Get Trade Type --list

In [137]:
tradetype = []
for i in above_table_empty_rows:
    # no need to plus 1 here
    tradetype.append(worksheet_now.cell(i+1,0).value)

### Get Firm name --list

In [138]:
firms = []
for i in range(n_cols):
    if check_not_empty(worksheet_now,above_table_empty_rows[0]-1,i):
        firms.append(worksheet_now.cell(above_table_empty_rows[0]-1,i).value)

###  tenors (and sizes) -- a dict  indexing on Trade Type --- and is same for each company

In [139]:
# different trade type may have different ternor collection
tenors = {}
size_bases = {}
frame_rows = above_table_empty_rows.copy()
frame_cols = empty_cols.copy()
frame_rows.append(effective_rows[-1]+1)
frame_cols.append(effective_cols[-1]+1)
j=0
# first loop: each descriptive table:higher to lower

for i in range(len(frame_rows)-1): 
    
    symbol = '+'
    row_inuse = frame_rows[i]
    next_row = frame_rows[i+1]
    tenor_ele = {}
    tenor_choice = []
    size_choice = []
    
    name = tradetype[j]
    
    for rows_in_table in range(row_inuse+2,next_row):
        a_tenor = ''
        a_size = 0
        temp_pair = []
        if rows_in_table == row_inuse+2:
            for k in range(1,empty_cols[0]):
                if check_not_empty(worksheet_now,rows_in_table,k):
                    tenor_ele[k] = worksheet_now.cell(rows_in_table,k).value
                    
        if rows_in_table != row_inuse+2:    
            for k in range(1,empty_cols[0]): # empty is in fact the furtherest 
                if check_not_empty(worksheet_now,rows_in_table,k):
                    if  worksheet_now.cell(rows_in_table,k).value >0:
                        symbol = '+'
                    else:
                        symbol = '-'
                    a_tenor= a_tenor + symbol + tenor_ele[k]+ ' '
                    a_size = worksheet_now.cell(rows_in_table,k).value
                    temp_pair.append(a_tenor)
                    temp_pair.append(abs(a_size)*1000000)
                    a_size = temp_pair
            tenor_choice.append(a_tenor)
            #size_choice.append(abs(a_size)*1000000)
            size_choice.append((a_size))
    tenors[name] = tenor_choice  
    size_bases[name] = size_choice
    #size_bases[name] = temp_pair
    j=j+1

### size multiplier; which indexing on tradetype

In [140]:
size_mul = {}
for i in above_table_empty_rows:
    multiplier = []
    name = worksheet_now.cell(i+1,0).value
    for j in range(empty_cols[0]+1,empty_cols[1]):
        if check_not_empty(worksheet_now,i+2,j):
            multiplier.append(get_size_from_string(worksheet_now.cell(i+2,j).value))
            #multiplier.append(worksheet_now.cell(i+2,j).value)
        else:pass;
    size_mul[name] = multiplier

### loop each company to find out the spread

In [141]:
output = pd.DataFrame(columns= ['CCY','Trade_Type','Firm','Tenor','DV01_Size','Bid_Ask_Spread'])

In [142]:
spreads = []
for j in range(len(frame_rows)-1):    
    row_inuse = frame_rows[j]
    #print(row_inuse)
    row_next = frame_rows[j+1]

    for i in range(len(frame_cols)-1):
        col_inuse = frame_cols[i]+1
        col_next = frame_cols[i+1]
        for arow in range(row_inuse+3,row_next):
            for acol in range(col_inuse,col_next):
                spreads.append(worksheet_now.cell(arow,acol).value)
                #print(arow,acol)
                

### prepare sizes

In [143]:
sizes = {}
for name in tradetype:
    l1 = size_bases[name]
    l2 = size_mul[name]
    temp_lista = []
    temp_listb = []
    for i in range(len(l1)):
        a_tenor = tenors[name][i]
        for j in range(len(l2)):
            _num = l1[i][1] # it is the number of base
            _mul = l2[j]
            temp_lista.append([_num*_mul,a_tenor])
    sizes[name] = temp_lista

In [144]:
#for ccy in sheet_names:
test=0
classify = []
temp_num = 0
for tt in tradetype:
    for firm in firms:
            for siz in sizes[tt]:
                li = [worksheet_now.name,tt,firm,siz[0],siz[1],spreads[temp_num]]
                temp_num = temp_num+1
                classify.append(li)

In [145]:
df_output = pd.DataFrame.from_records(classify)

In [146]:
df_output.to_csv('output_test')

In [147]:
df_output

,0,1,2,3,4,5
0,USD,Vanilla Outright,"Average of 5th, 6th and 7th smallest charge",2000000.0,+3m**,1.22222
1,USD,Vanilla Outright,"Average of 5th, 6th and 7th smallest charge",5000000.0,+3m**,2
2,USD,Vanilla Outright,"Average of 5th, 6th and 7th smallest charge",10000000.0,+3m**,3.66667
3,USD,Vanilla Outright,"Average of 5th, 6th and 7th smallest charge",20000000.0,+3m**,5
4,USD,Vanilla Outright,"Average of 5th, 6th and 7th smallest charge",50000000.0,+3m**,7.66667
5,USD,Vanilla Outright,"Average of 5th, 6th and 7th smallest charge",100000000.0,+3m**,12
6,USD,Vanilla Outright,"Average of 5th, 6th and 7th smallest charge",2000000.0,+6m**,1.66667
7,USD,Vanilla Outright,"Average of 5th, 6th and 7th smallest charge",5000000.0,+6m**,2.16667
8,USD,Vanilla Outright,"Average of 5th, 6th and 7th smallest charge",10000000.0,+6m**,4
9,USD,Vanilla Outright,"Average of 5th, 6th and 7th smallest charge",20000000.0,+6m**,5.66667
